In [1]:
import yaml
from phonemize import phonemize



In [2]:
import phonemizer
global_phonemizer = phonemizer.backend.EspeakBackend(language='en-us', preserve_punctuation=True,  with_stress=True, language_switch="remove-flags")

In [3]:
config_path = "Configs/config.yml" # you can change it to anything else
config = yaml.safe_load(open(config_path))

In [ ]:
from transformers import TransfoXLTokenizer

english_tokenizer =  TransfoXLTokenizer.from_pretrained("transfo-xl-wt103") # you can use any other tokenizers if you want to

In [5]:
root_directory = "/mnt/data/wiki_phoneme"

In [6]:
from datasets import load_from_disk, concatenate_datasets
import os 

output = [dI for dI in os.listdir(root_directory) if os.path.isdir(os.path.join(root_directory,dI))]
datasets = []
for o in output:
    directory = root_directory + "/" + o
    try:
        shard = load_from_disk(directory)
        datasets.append(shard)
        # print("%s loaded" % o)
    except:
        print("Continued")
        continue

In [7]:
english_dataset = concatenate_datasets(datasets)

In [ ]:
english_dataset

In [ ]:
from simple_loader import FilePathDataset, build_dataloader

english_file_data1 = FilePathDataset(english_dataset)
en_simple_loader = build_dataloader(english_file_data1, num_workers=32, batch_size=1)

In [ ]:
for batch in en_simple_loader:
    print(batch)
    break

In [132]:
# english_dataset[0]['phonemes']

In [ ]:
from dataloader import FilePathDataset, build_dataloader

english_file_data1 = FilePathDataset(dataset=english_dataset)
en_loader = build_dataloader(english_dataset, num_workers=32, batch_size=1)

In [ ]:
phonemes, words, labels, masked_index = english_file_data1[0]

In [ ]:
dd.token_maps[english_dataset[0]['input_ids'][3]]

In [ ]:
english_tokenizer.decode(english_dataset[0]['input_ids'][3])

In [ ]:
print

In [ ]:
for batch in en_loader:
    words, labels, phonemes, input_lengths, masked_indices = batch

    break

In [ ]:
len(english_dataset[0]['phonemes'])

In [ ]:
len(english_dataset[0]['input_ids'])

In [16]:
hindi_dataset = load_from_disk("/home/ubuntu/PL-BERT/data/multilingual-pl-bert/hi")

In [63]:
#coding: utf-8

import os
import os.path as osp
import time
import random
import numpy as np
import random

import string
import pickle

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from text_utils import TextCleaner
from utils import align_subtokens_to_phonemes

import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

np.random.seed(1)
random.seed(1)

class FilePathDataset(torch.utils.data.Dataset):
    def __init__(self, dataset,
                 token_maps="token_maps.pkl",
                 tokenizer="transfo-xl-wt103",
                 word_separator=3039, 
                 token_separator=" ", 
                 token_mask="M", 
                 max_mel_length=512,
                 word_mask_prob=0.15,
                 phoneme_mask_prob=0.1,
                 replace_prob=0.2):
        
        self.data = dataset
        self.max_mel_length = max_mel_length
        self.word_mask_prob = word_mask_prob
        self.phoneme_mask_prob = phoneme_mask_prob
        self.replace_prob = replace_prob
        self.text_cleaner = TextCleaner()
        
        self.word_separator = word_separator
        self.token_separator = token_separator
        self.token_mask = token_mask
        self.tokenizer = tokenizer
        
        with open(token_maps, 'rb') as handle:
            self.token_maps = pickle.load(handle)     
            
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        phonemes = self.data[idx]['phonemes']
        input_ids = self.data[idx]['input_ids']

        words = []
        labels = ""
        phoneme = ""
        print(phonemes)

        phoneme_list = ''.join(phonemes)
        masked_index = []
        for z in zip(phonemes, input_ids):
            z = list(z)
            
            if len(z[1]) == 1:
                words.extend([z[1][0]] * len(z[0]))
            else:
                subtokens = [self.tokenizer.decode(id) for id in z[1]]
                mapping = align_subtokens_to_phonemes(subtokens=subtokens, ipa_phonemes=z[0], token_ids=z[1])
                phonemes_idx = 0
                for subtoken, singl_phoneme, token_id in mapping:
                    assert z[0][phonemes_idx:phonemes_idx+len(singl_phoneme)] == singl_phoneme
                    phonemes_idx += len(singl_phoneme)
                    words.extend([token_id] * len(singl_phoneme))

            words.append(self.word_separator)
            labels += z[0] + " "

            if np.random.rand() < self.word_mask_prob:
                if np.random.rand() < self.replace_prob:
                    if np.random.rand() < (self.phoneme_mask_prob / self.replace_prob): 
                        phoneme += ''.join([phoneme_list[np.random.randint(0, len(phoneme_list))] for _ in range(len(z[0]))])  # randomized
                    else:
                        phoneme += z[0]
                else:
                    phoneme += self.token_mask * len(z[0]) # masked
                    
                masked_index.extend((np.arange(len(phoneme) - len(z[0]), len(phoneme))).tolist())
            else:
                phoneme += z[0] 

            phoneme += self.token_separator

        print("After Loop: ", phoneme)
        mel_length = len(phoneme)
        masked_idx = np.array(masked_index)
        masked_index = []
        if mel_length > self.max_mel_length:
            random_start = np.random.randint(0, mel_length - self.max_mel_length)
            phoneme = phoneme[random_start:random_start + self.max_mel_length]
            words = words[random_start:random_start + self.max_mel_length]
            labels = labels[random_start:random_start + self.max_mel_length]
            
            for m in masked_idx:
                if m >= random_start and m < random_start + self.max_mel_length:
                    masked_index.append(m - random_start)
        else:
            masked_index = masked_idx
        print(phoneme)
        phoneme = self.text_cleaner(phoneme)
        print(phoneme)
        labels = self.text_cleaner(labels)
        words = [self.token_maps[w]['token'] for w in words]
        
        assert len(phoneme) == len(words)
        assert len(phoneme) == len(labels)
        
        print(phoneme, words, labels)
        phonemes = torch.LongTensor(phoneme)
        labels = torch.LongTensor(labels)
        words = torch.LongTensor(words)
        
        return phoneme, words, labels, masked_index
        
class Collater(object):
    """
    Args:
      adaptive_batch_size (bool): if true, decrease batch size when long data comes.
    """

    def __init__(self, return_wave=False):
        self.text_pad_index = 0
        self.return_wave = return_wave
        

    def __call__(self, batch):
        # batch[0] = wave, mel, text, f0, speakerid
        batch_size = len(batch)

        # sort by mel length
        lengths = [b[1].shape[0] for b in batch]
        batch_indexes = np.argsort(lengths)[::-1]
        batch = [batch[bid] for bid in batch_indexes]

        max_text_length = max([b[1].shape[0] for b in batch])

        words = torch.zeros((batch_size, max_text_length)).long()
        labels = torch.zeros((batch_size, max_text_length)).long()
        phonemes = torch.zeros((batch_size, max_text_length)).long()
        input_lengths = []
        masked_indices = []
        for bid, (phoneme, word, label, masked_index) in enumerate(batch):
            
            text_size = phoneme.size(0)
            words[bid, :text_size] = word
            labels[bid, :text_size] = label
            phonemes[bid, :text_size] = phoneme
            input_lengths.append(text_size)
            masked_indices.append(masked_index)

        return words, labels, phonemes, input_lengths, masked_indices


def build_dataloader(df,
                     validation=False,
                     batch_size=4,
                     num_workers=1,
                     device='cpu',
                     collate_config={},
                     dataset_config={}):

    dataset = FilePathDataset(df, **dataset_config)
    collate_fn = Collater(**collate_config)
    data_loader = DataLoader(dataset,
                             batch_size=batch_size,
                             shuffle=(not validation),
                             num_workers=num_workers,
                             drop_last=(not validation),
                             collate_fn=collate_fn,
                             pin_memory=(device != 'cpu'))

    return data_loader


In [ ]:
from transformers import AutoTokenizer
multi_lingual_tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

In [ ]:
dd = FilePathDataset(hindi_dataset)

In [ ]:
def align_subtokens_to_phonemes(subtokens, ipa_phonemes, token_ids):
    """
    Align subtokens to IPA phonemes along with token IDs based on length and sound structure.
    
    Parameters:
    subtokens (list of str): List of subtokens for a word.
    ipa_phonemes (str): IPA phoneme string for the word.
    token_ids (list of int): List of token IDs for each subtoken.
    
    Returns:
    list of tuple: Each tuple contains a subtoken, its aligned IPA phoneme, and its token ID.
    """
    # Check if subtokens and token IDs are of equal length
    if len(subtokens) != len(token_ids):
        raise ValueError("Subtokens and token IDs must be of the same length.")
    
    # Step 1: Calculate approximate length of IPA phonemes for each subtoken
    ipa_length = len(ipa_phonemes)
    subtoken_lengths = [len(st.replace('##', '')) for st in subtokens]
    total_length = sum(subtoken_lengths)
    
    # Determine split points in the IPA phonemes based on subtoken lengths
    split_points = []
    accumulated = 0
    for length in subtoken_lengths:
        portion = (length / total_length) * ipa_length
        accumulated += portion
        split_points.append(int(round(accumulated)))  # Round to nearest index
    
    # Step 2: Split the IPA phoneme string at the determined points
    aligned_phonemes = []
    start = 0
    for end in split_points:
        aligned_phonemes.append(ipa_phonemes[start:end])
        start = end

    # Step 3: Create a mapping of each subtoken to its aligned IPA phoneme and token ID
    mapping = list(zip(subtokens, aligned_phonemes, token_ids))
    
    return mapping

# Updated data including token IDs
data = [
    (['र', '##ी', '##वा'], 'ɾˈiːʋaː', [891, 10914, 28960]),
    (['के'], 'keː', [10412]),
    (['ज', '##ंग', '##लों'], 'ɟˈʌŋɡəlˌõ', [872, 31222, 51665]),
    (['में'], 'mẽː', [10532]),
    (['ही'], 'hˈi', [14080]),
    (['स', '##फ', '##ेद'], 'səpʰˈeːd', [898, 28863, 82813]),
    (['ब', '##ा', '##घ'], 'bˈaːɡʰ', [887, 11208, 55759]),
    (['की'], 'ki', [10826]),
    (['न', '##स', '##्ल'], 'nˈʌslə', [884, 13432, 50101]),
    (['प', '##ाई'], 'pˈaːi', [885, 30472]),
    
    # New test data
    (['हैं'], 'hɛ̃', [11716]),
    (['।'], 'pˈuːrnwɪɾˈaːm', [920]),
    (['जिले'], 'ɟˈɪleː', [32291]),
    (['की'], 'ki', [10826]),
    (['प्रमुख'], 'pɾˈʌmʊkʰ', [29218]),
    (['उ', '##प', '##ज'], 'ˈʊpəɟ', [855, 18187, 17413]),
    (['ध', '##ान'], 'dʰˈaːn', [883, 21202]),
    (['है'], 'hɛː', [10569]),
    (['।'], 'pˈuːrnwɪɾˈaːm', [920]),
    (['जिले'], 'ɟˈɪleː', [32291]),
    (['के'], 'keː', [10412]),
    (['त', '##ाला'], 'tˈaːlaː', [880, 65986]),
    (['नामक'], 'nˈaːmək', [56734]),
    (['ज', '##ंग', '##ल'], 'ɟˈʌŋɡəl', [872, 31222, 11714])
]

# Run alignment for each word in the new test data
for subtokens, ipa_phonemes, token_ids in data:
    mapping = align_subtokens_to_phonemes(subtokens, ipa_phonemes, token_ids)
    print(f"IPA: {ipa_phonemes}, Subtokens: {subtokens}, Token IDs: {token_ids}")
    print("Mapping (Subtoken, Phoneme, Token ID):", mapping)
    print()

In [ ]:
phonemes = dd.data[0]['phonemes']
input_ids = dd.data[0]['input_ids']

words = []
labels = ""
phoneme = ""

phoneme_list = ''.join(phonemes)
masked_index = []
for z in zip(phonemes, input_ids):
    z = list(z)
    
    if len(z[1]) == 1:
        words.extend([z[1][0]] * len(z[0]))
    else:
        subtokens = [multi_lingual_tokenizer.decode(id) for id in z[1]]
        mapping = align_subtokens_to_phonemes(subtokens=subtokens, ipa_phonemes=z[0], token_ids=z[1])
        phonemes_idx = 0
        for subtoken, singl_phoneme, token_id in mapping:
            assert z[0][phonemes_idx:phonemes_idx+len(singl_phoneme)] == singl_phoneme
            phonemes_idx += len(singl_phoneme)
            words.extend([token_id] * len(singl_phoneme))

    words.append(dd.word_separator)
    labels += z[0] + " "

    if np.random.rand() < dd.word_mask_prob:
        if np.random.rand() < dd.replace_prob:
            if np.random.rand() < (dd.phoneme_mask_prob / dd.replace_prob): 
                phoneme += ''.join([phoneme_list[np.random.randint(0, len(phoneme_list))] for _ in range(len(z[0]))])  # randomized
            else:
                phoneme += z[0]
        else:
            phoneme += dd.token_mask * len(z[0]) # masked
            
        masked_index.extend((np.arange(len(phoneme) - len(z[0]), len(phoneme))).tolist())
    else:
        phoneme += z[0] 

    phoneme += dd.token_separator

mel_length = len(phoneme)
masked_idx = np.array(masked_index)
masked_index = []
if mel_length > dd.max_mel_length:
    random_start = np.random.randint(0, mel_length - dd.max_mel_length)
    phoneme = phoneme[random_start:random_start + dd.max_mel_length]
    words = words[random_start:random_start + dd.max_mel_length]
    labels = labels[random_start:random_start + dd.max_mel_length]
    
    for m in masked_idx:
        if m >= random_start and m < random_start + dd.max_mel_length:
            masked_index.append(m - random_start)
else:
    masked_index = masked_idx

print(phoneme[:10])
print(labels[:10])

phoneme = dd.text_cleaner(phoneme)
labels = dd.text_cleaner(labels)

# words = [dd.token_maps[w]['token'] for w in words]

# assert len(phoneme) == len(words)
# assert len(phoneme) == len(labels)

# print(phoneme, words, labels)
# phonemes = torch.LongTensor(phoneme)
# labels = torch.LongTensor(labels)
# words = torch.LongTensor(words)


In [ ]:
for w in words:
    print(dd.token_maps[w])

In [ ]:
dd.token_maps

In [ ]:
phoneme[:10]

In [ ]:
words[:10]

In [ ]:
labels[:10]

In [ ]:
phoneme

In [ ]:
phoneme, words, labels, masked_index = dd[0]

In [ ]:
phoneme[:10]

In [ ]:
labels[:10]

In [60]:
# for z in zip(hindi_dataset[0]['phonemes'], hindi_dataset[0]['input_ids']):
#     decoded = [multi_lingual_tokenizer.decode([token]) for token in z[1]]

#     if len(z[1]) == 1:
#         continue
#     # print(multi_lingual_tokenizer.decode(z[1]), ": ", z[0], z[1], decoded, len(z[0]), len(z[1]))
#     print("Mapping: ", align_subtokens_to_phonemes(subtokens=decoded, ipa_phonemes=z[0], token_ids=z[1]))

In [ ]:
hi_loader = build_dataloader(hindi_dataset, num_workers=32, batch_size=1)

In [ ]:
hi_loader

In [ ]:
for batch in hi_loader:
    words, labels, phonemes, input_lengths, masked_indices = batch

    print()
    break

In [ ]:
len(labels[0])